## Nan 처리하기

- Nan 을 처리할때, 결국 중요시 되는 feature들은 보험사기자들이 어떤 feature를 가지고 있는지 체크해봐야함. 일반고객의 수 > 보험사기자 수 이기 때문에 ==> 일반고객들의 null값을 살펴보기보다는 보험사기자들의 null값들이 어떻게 되어있는지 살펴본 후, 어떻게 빈칸을 처리할 것인지 판단하려고 함.

In [25]:
import pandas as pd
import glob
import numpy as np

In [41]:
merged_df_except_family = pd.read_csv('./Data/merged_df_except_family.csv', encoding='utf-16')

### 전처리 대상 Columns

In [42]:
# Null 값이 포함된 Column 찾기
columns = merged_df_except_family.columns
null_in_columns = []
for column in columns:
    result = merged_df_except_family[column].isnull().any()
    if result == True:
        null_in_columns.append(column)
print("< 전처리 대상 컬럼 리스트 >")
print(null_in_columns)
print(" ")
print("< 전처리 대상 컬럼 수 >")
print(len(null_in_columns))

< 전처리 대상 컬럼 리스트 >
['SIU_CUST_YN', 'RESI_TYPE_CODE', 'CUST_RGST', 'CTPR', 'OCCP_GRP_1', 'OCCP_GRP_2', 'TOTALPREM', 'MINCRDT', 'MAXCRDT', 'WEDD_YN', 'MATE_OCCP_GRP_1', 'MATE_OCCP_GRP_2', 'CHLD_CNT', 'LTBN_CHLD_AGE', 'MAX_PAYM_YM', 'MAX_PRM', 'CUST_INCM', 'JPBASE_HSHD_INCM', 'EXTN_YM', 'LAPS_YM', 'DISTANCE', 'ACCI_OCCP_GRP1', 'ACCI_OCCP_GRP2', 'HOSP_OTPA_STDT', 'HOSP_OTPA_ENDT', 'HOUSE_HOSP_DIST', 'HOSP_CODE', 'ACCI_HOSP_ADDR', 'HOSP_SPEC_DVSN', 'PAYM_DATE', 'SELF_CHAM', 'NON_PAY', 'TAMT_SFCA', 'PATT_CHRG_TOTA', 'DSCT_AMT', 'COUNT_TRMT_ITEM', 'DCAF_CMPS_XCPA', 'BEFO_JOB', 'EDGB']
 
< 전처리 대상 컬럼 수 >
39


### target 값을 제외한 38개의 컬럼
* SIU_CUST_YN : 보험사기자 구분 (X)
* RESI_TYPE_CODE : 주택유형 ==> 주택가격이 0이면 null ==> 없음으로 처리가능
* CUST_RGST : 고객등록일
* CTPR : 거주 주소
* OCCP_GRP_1 : 직업분류코드1 ==> 실업자의 경우 직업이 없을 수 있음
* OCCP_GRP_2 : 직업분류코드2 ==> 위와 동일
* TOTALPREM : 납입총보험료
* MINCRDT : 최소신용등급 ==> 미확인시 6등급 ==> 6등급으로 처리
* MAXCRDT : 최대신용등급 ==> 미확인시 6등급 ==> 6등급으로 처리
* WEDD_YN : 결혼여부
* MATE_OCCP_GRP_1 : 배우자직업코드1 ==> 결혼을 하지 않았으면 null
* MATE_OCCP_GRP_2 : 배우자직업코드2 ==> 위와동일
* CHLD_CNT : 자녀수 
* LTBN_CHLD_AGE : 막내자녀연령
* MAX_PAYM_YM : 최대보험료연월
* MAX_PRM : 최대보험료
* CUST_INCM : 고객의 추정소득
* JPBASE_HSHD_INCM : 가구의 추정소득2
* EXTN_YM : 소멸년월 (없으면 null) ==> 이건 어떻게 처리?
* LAPS_YM : 실효년월 (없으면 null) ==> 이건 어떻게 처리?
* DISTANCE : FP와 고객거주지의 거리 ==> 거주주소가 없으므로 거리도 나오지 않음
* ACCI_OCCP_GRP1 : 직업그룹코드1
* ACCI_OCCP_GRP2 : 직업그룹코드2
* HOSP_OTPA_STDT : 입원/통원시작일자
* HOSP_OTPA_ENDT : 입원/통원종료일자
* HOUSE_HOSP_DIST : 고객 거주지와 병원까지 거리
* HOSP_CODE : 병원코드
* ACCI_HOSP_ADDR : 병원지역(시도)
* HOSP_SPEC_DVSN : 병원종별구분
* PAYM_DATE : 지급일자
* SELF_CHAM : 본인부담금
* NON_PAY : 비급여
* TAMT_SFCA : 전액본인부담금
* PATT_CHRG_TOTA : 환자부담총액
* DSCT_AMT : 병원에서 할인해준 금액
* COUNT_TRMT_ITEM : 진료과목갯수
* DCAF_CMPS_XCPA : 실손상품에서 보상제외금액을 의미
* BEFO_JOB : FP 입사전 직업
* EDGB : FP 학력

* "#" 값의경우 정확한 내용을 알 수 없음으로 해석하면 됨

In [10]:
# "#" 값이 포함된 column 찾기
sharp_in_column = ['ETRS_YM', 'FIRE_YM', 'EDGB', 'CAUS_CODE_DTAL']
# merged_df_except_family['CAUS_CODE_DTAL'].str.contains('#').any()

* CAUS_CODE_DTAL : 원인코드상세
* ETRS_YM : FP 입사년월
* FIRE_YM : FP 퇴사년월
* EDGB : FP 학력
    - '#' 값 = '정확히 알수없음'을 의미

## How to deal with Null in column

In [24]:
# 'RESI_TYPE_CODE' : 주택유형 ( C )
# nan => 주택없음 (10) 으로 처리
merged_df_except_family['RESI_TYPE_CODE'].unique()

array([ 20.,  13.,  99.,  30.,  nan,  11.,  40.,  12.,  50.,  60.,  70.])

In [21]:
merged_df_except_family['RESI_TYPE_CODE'].fillna(value=10, inplace=True)
merged_df_except_family['RESI_TYPE_CODE'].unique()

array([ 20.,  13.,  99.,  30.,  10.,  11.,  40.,  12.,  50.,  60.,  70.])

In [32]:
# 'CUST_RGST' : 고객등록일 ( C )
# nan => '#' 으로 처리
merged_df_except_family['CUST_RGST'].unique().shape

(373,)

In [27]:
merged_df_except_family['CUST_RGST'].fillna(value='#', inplace=True)
is_sharp = merged_df_except_family['CUST_RGST'] == '#'
merged_df_except_family[is_sharp]['CUST_RGST'].shape

(3550,)

In [39]:
# 'CTPR' : 시도구분 ( C )
# nan => '#' 으로 처리
merged_df_except_family['CTPR'].unique().shape

(18,)

In [28]:
merged_df_except_family['CTPR'].fillna(value='#', inplace=True)
merged_df_except_family['CTPR'].unique()

array(['충북', '서울', '부산', '광주', '충남', '강원', '경남', '전북', '#', '경기', '경북',
       '인천', '대전', '전남', '울산', '대구', '제주', '세종'], dtype=object)

In [75]:
# 'OCCP_GRP_1' : 직업분류코드1 ( C )
# 'OCCP_GRP_2' : 직업분류코드2 ( C )
# nan이 있는 경우 ==> 코드1 과 코드2가 모두 비어있음 ==> 실업자(0. 실업자)로 처리
grp_1 = merged_df_except_family['OCCP_GRP_1'].unique().shape
grp_2 = merged_df_except_family['OCCP_GRP_2'].unique().shape
print(grp_1, grp_2)

(9,) (26,)


In [41]:
merged_df_except_family['OCCP_GRP_1'].fillna(value='0.해당없음', inplace=True)
merged_df_except_family['OCCP_GRP_2'].fillna(value='해당없음', inplace=True)
grp_1 = merged_df_except_family['OCCP_GRP_1'].unique()
grp_2 = merged_df_except_family['OCCP_GRP_2'].unique()
print(grp_1, grp_2)

['3.사무직' '4.전문직' '8.기타' '0.실업자' '5.서비스' '2.자영업' '1.주부' '6.제조업' '7.1차산업'] ['사무직' '의료직 종사자' '전문직' '고소득 전문직' '학생' '실업자' '2차산업 종사자' '3차산업 종사자' '고위 공무원'
 '예체능계 종사자' '주부' '자영업' '공무원' '교사' '학자/연구직' '교육관련직' '기타' '기업/단체 임원' '단순 사무직'
 '대학교수/강사' '운전직' '단순 노무직' '1차산업 종사자' '종교인/역술인' '고소득의료직' '법무직 종사자']


#### Not now

In [34]:
# 'TOTALPREM' : 납입총보험료 ( N )
# nan => drop 하는게 낫지 않을까 or 직업코드에 따른 평균치를 넣어볼까?

# < Workflow >
# 1. grp1 or grp2 가 같은 사람들끼리 납입총보험료를 구하여 평균을 구함
# 2. 같은 직업코드인 사람중에 nan 이 있는경우 평균값을 집어넣어라.

In [49]:
# 'MINCRDT' : 최소신용등급 ( C )
# 'MAXCRDT' : 최대신용등급 ( C )
# nan => '6' 등급으로 처리
mincrdt = merged_df_except_family['MINCRDT'].unique().shape
maxcrdt = merged_df_except_family['MAXCRDT'].unique().shape
print(mincrdt, maxcrdt)

(14,) (14,)


In [37]:
merged_df_except_family['MINCRDT'].fillna(value=6, inplace=True)
merged_df_except_family['MAXCRDT'].fillna(value=6, inplace=True)

mincrdt = merged_df_except_family['MINCRDT'].unique()
maxcrdt = merged_df_except_family['MAXCRDT'].unique()
print(mincrdt, maxcrdt)
# 여기서 99 나 28같은경우도 미확인된것이므로 6으로 처리하면 될까?

[  6.   1.   2.   3.  10.   8.   5.   4.   7.   0.   9.  99.  28.] [  6.  99.  10.   8.   4.   2.   7.   9.   0.   3.  28.   1.   5.]


In [50]:
# WEDD_YN : 결혼여부 ( C )
# nan => '#' 으로 처리
merged_df_except_family['WEDD_YN'].unique()

array(['Y', 'N', nan], dtype=object)

In [38]:
merged_df_except_family['WEDD_YN'].fillna(value='#', inplace=True)
merged_df_except_family['WEDD_YN'].unique()

array(['Y', 'N', '#'], dtype=object)

#### Not now

In [47]:
# 'MATE_OCCP_GRP_1' : 배우자직업코드1 ( C )
# 'MATE_OCCP_GRP_2' : 배우자직업코드2 ( C )
# nan => if wedd_yn = N -> 결혼을 하지 않은 사람 ( 0.해당없음 ) / 
#                     # -> 알수없음 ( # )

# if merged_df_except_family['WEDD_YN'] == 'N':
#     merged_df_except_family['MATE_OCCP_GRP_1'].fillna(value='0.해당없음')
#     merged_df_except_family['MATE_OCCP_GRP_2'].fillna(value='해당없음')

# if merged_df_except_family['WEDD_YN'] == '#':
#     merged_df_except_family['MATE_OCCP_GRP_1'].fillna(value='#')
#     merged_df_except_family['MATE_OCCP_GRP_2'].fillna(value='#')

In [7]:
# 'CHLD_CNT' : 자녀수 ( N )
# 'LTBN_CHLD_AGE' : 막내자녀연령 ( N )
# nan => 0 으로 처리
chld = merged_df_except_family['CHLD_CNT'].unique()
last_chld_age = merged_df_except_family['LTBN_CHLD_AGE'].unique()
print(chld)
print(last_chld_age)

[ 2.  1.  0.  3.  4.  5.  6.]
[ 13.  17.   9.  23.  14.  10.   0.   5.  21.   6.  20.   4.  19.  34.  31.
  26.  32.  15.  11.  nan   8.  35.  12.  28.  29.  22.  18.  24.  16.  27.
  25.   3.  30.  33.   7.   2.   1.]


In [8]:
merged_df_except_family['CHLD_CNT'].fillna(value=0, inplace=True)
merged_df_except_family['CHLD_CNT'].unique()

array([ 2.,  1.,  0.,  3.,  4.,  5.,  6.])

In [10]:
merged_df_except_family['LTBN_CHLD_AGE'].fillna(value=0, inplace=True)
merged_df_except_family['LTBN_CHLD_AGE'].unique()

array([ 13.,  17.,   9.,  23.,  14.,  10.,   0.,   5.,  21.,   6.,  20.,
         4.,  19.,  34.,  31.,  26.,  32.,  15.,  11.,   8.,  35.,  12.,
        28.,  29.,  22.,  18.,  24.,  16.,  27.,  25.,   3.,  30.,  33.,
         7.,   2.,   1.])

In [55]:
# 'MAX_PAYM_YM' : 최대보험료연월 ( C )
# nan => '#' 으로 처리

#### Not now

In [57]:
# 'MAX_PRM' : 최대보험료 ( N )
# nan => 직업코드에 따른 평균값으로 처리하면 어떨지...?
# 약 30% 정도가 비어있음 (16000 / 22500)

#### Not now

In [58]:
# 'CUST_INCM' : 고객의 추정소득 ( N )
# nan => 연령/직업/보험료수준 에 따른 평균값?

#### Not now

In [60]:
# 'JPBASE_HSHD_INCM : 가구의 추정소득2 ( N )
# nan => 직업 및 납입보험료 수준 에 따른 ...?

In [62]:
# 'EXTN_YM' : 소멸년월 ( C )
# 'LAPS_YM' : 실효년월 ( C )
# nan => 소멸/실효 된적이 없으면 null ==> 해당없음(999999) 으로 처리

#### Not now

In [69]:
# 'DISTANCE' : FP와 고객거주지의 거리 ( N )
# nan => 이건 어떻게 처리해야할까...? 
# merged_df_except_family['DISTANCE']

In [63]:
# 'ACCI_OCCP_GRP1' : 직업그룹코드1 ( C )
# 'ACCI_OCCP_GRP2' : 직업그룹코드2 ( C )
# => 2개의 feature는 삭제 예정 ( 이유 : 직업분류코드와 중복 )

#### Not now

In [66]:
# 'HOSP_OTPA_STDT' : 입원/통원시작일자 ( C )
# 'HOSP_OTPA_ENDT' : 입원/통원종료일자 ( C )
# nan => '#' 으로 처리
# 그런데, 일자를 class 로 받게 되는것이 과연 실효성이 있을지 의문이 든다.
merged_df_except_family['HOSP_OTPA_STDT'].unique().shape

(3824,)

#### Not now

In [70]:
# 'HOUSE_HOSP_DIST' : 고객 거주지와 병원까지 거리 ( N )
# nan => 이건 어떻게 처리해야할까...? 

In [72]:
# 'HOSP_CODE' : 병원코드 ( C )
# nan => # 으로 처리
merged_df_except_family['HOSP_CODE'].unique().shape

(12535,)

In [73]:
# 'ACCI_HOSP_ADDR' : 병원지역(시도) ( C )
# nan => # 으로 처리

In [74]:
# 'HOSP_SPEC_DVSN' : 병원종별구분 ( C )
# nan => # 으로 처리

#### Not now

In [77]:
# 'PAYM_DATE' : 지급일자 ( C )
# nan => 이건 어떻게 처리할까??
# merged_df_except_family['PAYM_DATE']

In [80]:
# 'SELF_CHAM' : 본인부담금 ( N )
# 'NON_PAY' : 비급여 ( N )
# 'TAMT_SFCA' : 전액본인부담금 ( N )
# PATT_CHRG_TOTA : 환자부담총액 ( N )
# DSCT_AMT : 병원에서 할인해준 금액 ( N )
# COUNT_TRMT_ITEM : 진료과목갯수 ( N )
# DCAF_CMPS_XCPA : 실손상품에서 보상제외금액을 의미 ( N )
# if 실손처리여부 == N -> 모두 0 으로 처리하자.

In [1]:
# 'BEFO_JOB' : FP 입사전 직업 ( C )
# 'EDFB' : FP 학력 ( C )
# nan => # 으로 처리하자.

In [27]:
def print_columns_containing_null(dataframe):
    columns = dataframe.columns
    null_in_columns = []
    for column in columns:
        result = dataframe[column].isnull().any()
        if result == True:
            null_in_columns.append(column)
    print("< 전처리 대상 컬럼 리스트 >")
    print(null_in_columns)
    print(" ")
    print("< 전처리 대상 컬럼 수 >")
    print(len(null_in_columns))

In [47]:
# target_columns_list = [
#     'RESI_TYPE_CODE', 'CUST_RGST', 'CTPR', 'OCCP_GRP_1', 'OCCP_GRP_2',
#     'MINCRDT', 'MAXCRDT', 'WEDD_YN', 'MATE_OCCP_GRP_1', 'MATE_OCCP_GRP_2',
#     'CHLD_CNT', 'LTBN_CHLD_AGE', 'MAX_PAYM_YM', 'MAX_PRM', 'CUST_INCM',
#     'JPBASE_HSHD_INCM', 'EXTN_YM', 'LAPS_YM', 'DISTANCE', 'ACCI_OCCP_GRP1',
#     'ACCI_OCCP_GRP2', 'HOSP_OTPA_STDT', 'HOSP_OTPA_ENDT', 'HOUSE_HOSP_DIST',
#     'HOSP_CODE', 'ACCI_HOSP_ADDR', 'HOSP_SPEC_DVSN', 'PAYM_DATE',
#     'SELF_CHAM', 'NON_PAY', 'TAMT_SFCA', 'PATT_CHRG_TOTA', 'DSCT_AMT',
#     'COUNT_TRMT_ITEM', 'DCAF_CMPS_XCPA', 'BEFO_JOB', 'EDGB'
# ]

for column in null_in_columns:
    target_values = merged_df_except_family[column]
    
    if column == 'RESI_TYPE_CODE': # 주택유형(C)
        target_values.fillna(value=10, inplace=True) # 주택없음(10)
    
    if column == 'CUST_RGST': # 고객등록일(C)
        target_values.fillna(value='#', inplace=True)
        
    if column == 'CTPR': # 시도구분(C)
        target_values.fillna(value='#', inplace=True)
        
    if column == 'OCCP_GRP_1': # 직업분류코드1(C)
        target_values.fillna(value='0.없음', inplace=True)
        
    if column == 'OCCP_GRP_2': # 직업분류코드2(C)
        target_values.fillna(value='없음', inplace=True)
        
    if column == 'TOTALPREM': # 납입총보험료(N)
        pass
    
    if column == 'MINCRDT' or 'MAXCRDT': # 최소/최대신용등급(C)
        target_values.fillna(value=6, inplace=True)
    
    if column == 'WEDD_YN': # 결혼여부(C)
        target_values.fillna(value='#', inplace=True)
        
    if column == 'MATE_OCCP_GRP_1' or 'MATE_OCCP_GRP2': # 배우자직업코드(C)
        pass
    
    if column == 'CHLD_CNT' or 'LTBN_CHLD_AGE': # 자녀수(N)/막내자녀연령(N)
        pass
    
    if column == 'MAX_PAYM_YM': # 최대보험료연월(C)
        target_values.fillna(value='#', inplace=True)
        
    if column == 'MAX_PRM': # 최대보험료(N)
        pass
    
    if column == 'CUST_INCM': # 고객추정소득(N)
        pass
    
    if column == 'JPBASE_HSHD_INCM': # 가구추정소득2(N)
        pass
    
    if column == 'EXTN_YM' or 'LAPS_YM': # 소멸/실효년월(C)
        target_values.fillna(value=999999, inplace=True)
        
    if column == 'DISTANCE': # FP와 고객거주지간 거리(N)
        pass
    
    if column == 'ACCI_OCCP_GRP1' or 'ACCI_OCCP_GRP2': # 직업그룹코드(C)
        pass
    
    if column == 'HOSP_OTPA_STDT' or 'HOSP_OTPA_ENDT': # 입원/통원시작/종료일자(C)
        pass
    
    if column == 'HOUSE_HOSP_DIST': # 병원과 고객거주지간 거리(N)
        pass
    
    if column == 'HOSP_CODE': # 병원코드(C)
        target_values.fillna(value='#', inplace=True)
        
    if column == 'ACCI_HOSP_ADDR': # 병원지역(시도)(C)
        target_values.fillna(value='#', inplace=True)
        
    if column == 'HOSP_SPEC_DVSN': # 병원종별구분(C)
        target_values.fillna(value='#', inplace=True)
        
    if column == 'PAYM_DATE': # 지급일자(C)
        pass
    
    if column == 'SELF_CHAM' or 'NON_PAY' or 'TAMT_SFCA' or \
            'TAMT_SFCA' or 'DSCT_AMT' or 'COUNT_TRMT_ITEM' or 'DCAF_CMPS_XCPA':
        pass 
    
    if column == 'BEFO_JOB' or 'EDGB': # FP입사전직업(C)/FP학력(C)
        target_values.fillna(value='#', inplace=True)

In [48]:
print_columns_containing_null(merged_df_except_family)

< 전처리 대상 컬럼 리스트 >
[]
 
< 전처리 대상 컬럼 수 >
0


In [45]:
merged_df_except_family['SELF_CHAM'].isnull().any()

False

In [46]:
merged_df_except_family.to_csv('./check.csv', encoding='utf-16')